In [1]:
import pandas as pd
import re
import pickle
import string

In [2]:
Dataset = 'Clotho' # or 'Clotho'

In [3]:
def repair_caption(caption) :
    caption = caption.replace(',', ' , ')
    caption = re.sub(' +', ' ', caption)
    caption = caption.replace(' ,', ',')

    return caption

In [4]:
def get_word_dictionary(caption_list) :
    word_dictionary = {}

    for caption in caption_list :

        caption = caption.lower()
        
        # 쉼표를 '안녕,하세요'와 같이 쓰는 사례가 많더라
        # 얘들을 다 교정해준다 
        # 교정========================================
        caption = caption.replace(',', ' , ') 
        # 공백 줄이기
        caption = re.sub(' +', ' ', caption)
        caption = caption.replace(' ,', ',')
        # 교정========================================

        # For ACT Tokenizer
        # caption = re.sub(r'\s([,.!?;:"](?:\s|$))', r'\1', caption).replace('  ', ' ')
        # caption = re.sub('[,.!?;:\"]', ' ', caption).replace('  ', ' ')

        # For Clotho tokenizer
        # caption = caption.translate(str.maketrans('', '', string.punctuation))

        # For Custom Tokenizer1
        caption = re.sub(r'[.]', '', caption)
        caption += '.'

        # For Custom Tokenizer2
        # caption = re.sub(r'[.]', '', caption)

        word_list = caption.strip().split(' ')

        for word in word_list : 
            
            # # Custom Tokenizer2의 경우 사용================================
            # if word in word_dictionary: 
            #     if word[-1] == ',' :
            #         temp_word_wo_rest = word[:-1] # 쉼표 있으면 쉼표 제거
            #         word_dictionary[temp_word_wo_rest] += 1
            #         word_dictionary[','] += 1 # 쉼표를 따로 추가
            #     else :
            #         word_dictionary[word] += 1
            # else : 
            #     if word[-1] == ',' :
            #         temp_word_wo_rest = word[:-1] # 쉼표 있으면 쉼표 제거
            #         word_dictionary[temp_word_wo_rest] = 1
            #         word_dictionary[','] = 1
            #     else :
            #         word_dictionary[word] = 1
            # # Custom Tokenizer2의 경우 사용================================

            if word in word_dictionary: 
                word_dictionary[word] += 1
            else : 
                word_dictionary[word] = 1
                
    return sorted(word_dictionary.items(), key = lambda item: item[1], reverse = True)

In [5]:
caption_list_audiocaps = []

train_csv = pd.read_csv('train.csv')

captions = train_csv['caption'].to_list()
for caption in captions:

    edited_caption = repair_caption(caption)

    # if edited_caption != caption :
    #     print("before repair :", caption)
    #     print("after repair :", edited_caption)
    #     print()

    caption_list_audiocaps.append(edited_caption)


In [6]:
word_dictionary_sorted = get_word_dictionary(caption_list_audiocaps) 

word_Dict = []

# 단어만 담기
# 그러면 등장 빈도수가 높은 단어부터 나열되는 딕셔너리가 완성됨
for key, value in word_dictionary_sorted :
    word_Dict.append(key)

print(len(word_Dict))

7908


In [7]:
# GPT-2는 vocab의 맨 첫 idx의 값을 "!"로 했다. 나도 그렇게 해야겠다.
# 그리고 GPT-2는 문장의 마무리라 할 수 있는 마침표('.')의 idx를 13으로 했다. 나는 문장의 마무리를 <eos>로 해야겠다. 
word_Dict.insert(0, '!')
word_Dict.insert(13, '<eos>')
word_Dict.append('<unk>')

In [8]:
# Clotho Dataset에만 있는 word들을 추가해줌
caption_list_clotho = []
if Dataset == 'Clotho' :
    train_csv = pd.read_csv('clotho_captions_development.csv')

    for i in range(5) :
        col_str = 'caption_' + str(i + 1)
    
        captions = train_csv[col_str].to_list()
        for caption in captions:

            edited_caption = repair_caption(caption)

            # if edited_caption != caption :
            #     print("before repair :", caption)
            #     print("after repair :", edited_caption)
            #     print()

            caption_list_clotho.append(caption)

    word_dictionary_sorted = get_word_dictionary(caption_list_clotho)
    # 단어만 담기
    # 그러면 등장 빈도수가 높은 단어부터 나열되는 딕셔너리가 완성됨
    not_in_count = 0
    in_count = 0
    for key, value in word_dictionary_sorted :

        if key not in word_Dict :
            word_Dict.append(key)
            not_in_count +=1
        else :
            in_count +=1
    
    print(str(not_in_count), "words not in audiocaps vocab")
    print(str(in_count), "words in audiocaps vocab")

2716 words not in audiocaps vocab
4292 words in audiocaps vocab


In [39]:
file_name = None
if Dataset == 'AudioCaps' :
    file_name = 'AudioCaps_vocabulary_'
elif Dataset == 'Clotho' :
    file_name = 'Clotho_vocabulary_'

file_name += (str(len(word_Dict)) + '.pickle')

with open(file_name, 'wb') as f:
    pickle.dump(word_Dict, f, pickle.HIGHEST_PROTOCOL)

In [9]:
len(word_Dict)

10627

In [15]:
word_Dict[7911]

'.'

In [11]:
with open('before_AudioCaps_vocabulary_7911.pickle', 'rb') as f:
    before_AudioCaps_vocabulary_7911 = pickle.load(f)

with open('AudioCaps_vocabulary_7911.pickle', 'rb') as f:
    AudioCaps_vocabulary_7911 = pickle.load(f)

In [12]:
len(before_AudioCaps_vocabulary_7911)

7911

In [13]:
len(AudioCaps_vocabulary_7911)

7911

In [14]:
for i in range(len(AudioCaps_vocabulary_7911)) :
    if AudioCaps_vocabulary_7911[i] != before_AudioCaps_vocabulary_7911[i] :
        print(before_AudioCaps_vocabulary_7911[i], AudioCaps_vocabulary_7911[i])

In [ ]:
with open('AudioCaps_vocabulary_7911.pickle', 'rb') as f:
    AudioCaps_vocabulary_7911 = pickle.load(f)

In [11]:
with open('Clotho_vocabulary_10640.pickle', 'rb') as f:
    Clotho_vocabulary_10640 = pickle.load(f)

In [14]:
for i in range(len(Clotho_vocabulary_10640)) :
    if word_Dict[i] != Clotho_vocabulary_10640[i] :
        print(word_Dict[i], Clotho_vocabulary_10640[i], i)

. book 7911
book proximity 7912
proximity night. 7913
night. travelling 7914
travelling gutter 7915
gutter cup. 7916
cup. wearing 7917
wearing house. 7918
house. hall 7919
hall thin 7920
thin cup 7921
cup stations 7922
stations stairs 7923
stairs upon 7924
upon any 7925
any forest. 7926
forest. restaurant 7927
restaurant kettle 7928
kettle approached 7929
approached drove 7930
drove heels 7931
heels stairs. 7932
stairs. create 7933
create branches 7934
branches coin 7935
coin cicadas 7936
cicadas printing 7937
printing flipped 7938
flipped elevator 7939
elevator erratic 7940
erratic garage. 7941
garage. market 7942
market volumes 7943
volumes throwing 7944
throwing wooded 7945
wooded conveyor 7946
conveyor walked 7947
walked factory. 7948
factory. porch 7949
porch organ 7950
organ adjusted 7951
adjusted emitted 7952
emitted swings 7953
swings tea 7954
tea book. 7955
book. pond. 7956
pond. tuner 7957
tuner sharpened 7958
sharpened hall. 7959
hall. hallway 7960
hallway overhead, 7961
ove

IndexError: list index out of range

In [22]:
word_Dict[7911:7921]

['.',
 'book',
 'proximity',
 'night.',
 'travelling',
 'gutter',
 'cup.',
 'wearing',
 'house.',
 'hall']

In [21]:
Clotho_vocabulary_10640[7911:7920]

['book',
 'proximity',
 'night.',
 'travelling',
 'gutter',
 'cup.',
 'wearing',
 'house.',
 'hall']